<a href="https://colab.research.google.com/github/janithcyapa/SmartHVAC/blob/main/Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install psychrolib

  Preparing metadata (setup.py) ... done
  Created wheel for psychrolib: filename=PsychroLib-2.5.0-py3-none-any.whl size=9766 sha256=cef3e582859db76ab9fcb6f282aacbbdb354d8957d9a5eec1c0f984d594624d5
  Stored in directory: /root/.cache/pip/wheels/db/56/d2/160fe5cd010497f465b750b273cc5596f7dbbd139622428d11
Successfully built psychrolib


In [3]:
import psychrolib as psy
import numpy as np

In [6]:
# Use SI units to standard
psy.SetUnitSystem(psy.SI)

In [9]:
N_Occ = 3 # Number of Occupants in the room
Q_Equ = 1000 # W - Sensible Heat from Equipment/Lights
T_room = 23.9 # °C - Initial dry bulb temperature
RH_room = 0.5 # 0 ~ 1 - Initial reletive humidity
P_room = 101325        # Pa -  Atmospheric pressure (sea level)
T_out = 35 # °C - Outside temperature
RH_out = 0.6 # 0 ~ 1 - Outside reletive humidity
L_room = 8 # m
A_wall =
A_window =

# Standard Definitions
# 1.0 Btu/h = 0.293 W
# Sensible Heat from Occupants
Q_S_Per_Occ = 200 * 0.293 # (W) - Typically 200 Btu/h per person for light activity
#Latent Heat from Occupants
Q_L_Per_Occ = 180 * 0.293 # (W) - Typically 180 Btu/h per person for light activity
U_wall =
U_window =

# Room Conditions
# Humidity ratio
W_room = psy.GetHumRatioFromRelHum(T_room, RH_room, P_room) # kg water/kg dry air
# Enthalpy
h_room_guess = psy.GetMoistAirEnthalpy(T_room, W_room ) # J/kg

# Calculate Total Room Sensible and Latent Heat Loads
Q_S_envelope = U_wall * A_wall  * (T_out - T_room) + U_window * A_window  * (T_out - T_room) # (W)
Q_S_total = Q_S_Per_Occ + Q_Equ + Q_S_envelope # (W) - add Q_solar_sensible if applicable
